# Lab-11-2 RNN hihello and charseq

In [1]:
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

## Data

In [2]:
sample = "can anybody find me somebody to love?"

In [3]:
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
char_dic

{'f': 0,
 ' ': 1,
 'v': 2,
 'e': 3,
 'o': 4,
 'l': 5,
 'd': 6,
 'i': 7,
 '?': 8,
 'a': 9,
 'm': 10,
 'b': 11,
 'c': 12,
 'y': 13,
 's': 14,
 't': 15,
 'n': 16}

In [4]:
dic_size = len(char_dic)

In [5]:
sample_idx = [char_dic[c] for c in sample]
sample_idx

[12,
 9,
 16,
 1,
 9,
 16,
 13,
 11,
 4,
 6,
 13,
 1,
 0,
 7,
 16,
 6,
 1,
 10,
 3,
 1,
 14,
 4,
 10,
 3,
 11,
 4,
 6,
 13,
 1,
 15,
 4,
 1,
 5,
 4,
 2,
 3,
 8]

In [6]:
x_data = [sample_idx[:-1]]
x_data

[[12,
  9,
  16,
  1,
  9,
  16,
  13,
  11,
  4,
  6,
  13,
  1,
  0,
  7,
  16,
  6,
  1,
  10,
  3,
  1,
  14,
  4,
  10,
  3,
  11,
  4,
  6,
  13,
  1,
  15,
  4,
  1,
  5,
  4,
  2,
  3]]

In [7]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
x_one_hot

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0.

In [8]:
y_data = [sample_idx[1:]]
y_data

[[9,
  16,
  1,
  9,
  16,
  13,
  11,
  4,
  6,
  13,
  1,
  0,
  7,
  16,
  6,
  1,
  10,
  3,
  1,
  14,
  4,
  10,
  3,
  11,
  4,
  6,
  13,
  1,
  15,
  4,
  1,
  5,
  4,
  2,
  3,
  8]]

In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

## Hyperparameters

In [10]:
input_size = dic_size
hidden_size = dic_size
learning_rate = 1e-1

## Model

In [11]:
rnn = nn.RNN(input_size, hidden_size, batch_first=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

## Train

In [12]:
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    
    if epoch % 10 == 0:
        print('Epoch:', epoch, 'Loss:', loss.item())
        print('Prediction:', result, 'Pred str:', result_str)

Epoch: 0 Loss: 2.8473989963531494
Prediction: [[16 16 16 16 16 16 16 16 16 16  4  8 16 16 16 16  5 11 16 16 16 16 16 16
  16 16 16 16 16  5 16 16  9  5  3 16]] Pred str: nnnnnnnnnno?nnnnlbnnnnnnnnnnnlnnalen
Epoch: 10 Loss: 1.6486926078796387
Prediction: [[13 16  1  9  1  5 11  4  6  1  1  0  7 16  6  1 10  3  1 14  4 10  3  1
   4  6  1  1  0  4  1  5 13  1  3  8]] Pred str: yn a lbod  find me some od  fo ly e?
Epoch: 20 Loss: 1.4857354164123535
Prediction: [[ 9 16 13  9 16 13 11  0  6 13  1  0  7  0  6  1 10  3  1 14  4  1  3 11
   4  6 13  1  0  4  1  5  4  1  3  8]] Pred str: anyanybfdy fifd me so ebody fo lo e?
Epoch: 30 Loss: 1.39260733127594
Prediction: [[ 9 16  1  9 16 13 11  0  6  1  1  0  7  0  6  1 10  3  1 14  4 10  3  1
   4  6  1  1  0  4  1  5  4  1  3  8]] Pred str: an anybfd  fifd me some od  fo lo e?
Epoch: 40 Loss: 1.3385896682739258
Prediction: [[ 9 16 13  9 16 13 11  4  6 13  1  0  7  0  6 13 10  3  1 14  4 10  3  1
   4  6 13  1  0  4  1  5  4  1  3  8]] Pred str: 